In [ ]:
import pandas as pd
import os

In [ ]:
# Get the current working directory
current_directory = os.getcwd()

# Name of CSV file
csv_filename = "processed-real-estate-listings.csv" 

# Combine the current directory path with the CSV file name to get the full file path
dataset_path = os.path.join(current_directory, csv_filename)

df = pd.read_csv(dataset_path)

In [ ]:
# Reorder columns so that "id" column becomes the first column
columns_order = ['id'] + [col for col in df.columns if col != 'id']
df = df[columns_order]

# Save the modified dataset
df.to_csv(dataset_path, index=False)